In [1]:
%pip install -q gurobipy
%pip install -q pandas
%pip install -q numpy
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
df = pd.read_csv("student_data.csv")

#Extract the number of students from the dataframe
NUM_STUDENTS = len(df)

#Create a list of students from 0 to num_students-1 for indexing purposes
students = list(range(num_students))

#compute number of teams from 0 to num_teams-1 for indexing purposes
NUM_TEAMS = len(students)//2

df.describe()

,Student,Preference 1,Preference 2,Preference 3
count,37.000000,37.000000,37.000000,37.000000
mean,18.000000,15.594595,16.054054,20.729730
std,10.824355,11.422345,11.217412,9.509202
min,0.000000,1.000000,0.000000,1.000000
25%,9.000000,5.000000,9.000000,15.000000
50%,18.000000,13.000000,13.000000,23.000000
75%,27.000000,26.000000,24.000000,28.000000
max,36.000000,36.000000,36.000000,36.000000


In [9]:
# Create preferences dictionary from dataframe
preferences = {}
for i in students:
    preferences[i] = df.iloc[i,1:6].tolist()
print(preferences)

{0: [1, 10, 22], 1: [7, 2, 17], 2: [19, 27, 15], 3: [4, 24, 14], 4: [36, 0, 23], 5: [21, 33, 25], 6: [8, 3, 29], 7: [28, 16, 1], 8: [11, 14, 6], 9: [35, 16, 24], 10: [2, 12, 31], 11: [8, 22, 28], 12: [10, 21, 15], 13: [34, 28, 17], 14: [3, 8, 27], 15: [2, 17, 12], 16: [7, 9, 31], 17: [13, 1, 2], 18: [26, 11, 22], 19: [2, 35, 25], 20: [34, 13, 9], 21: [12, 5, 36], 22: [32, 0, 33], 23: [4, 32, 20], 24: [3, 9, 27], 25: [30, 5, 19], 26: [18, 35, 7], 27: [14, 2, 24], 28: [11, 13, 7], 29: [31, 33, 6], 30: [25, 36, 35], 31: [16, 10, 29], 32: [22, 23, 30], 33: [5, 10, 29], 34: [13, 20, 28], 35: [26, 9, 19], 36: [4, 30, 23]}


In [8]:
m = gp.Model("student-teams")

# Create variables
# variable x[i,j] is 1 if student i is in team j and 0 otherwise
# variable y[j] is 1 if team j has at least 2 students and 0 otherwise
# there are at most as many teams as number of students / 2
NUM_TEAMS = len(students)//2
x = m.addVars(students, range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="x")
y = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="y")

# We create an auxiliary variable s[j] that is the score of team j
# The score of a team is the sum of the comfort of each student in each category
# We then maximize the minimum of the scores of all teams
s = m.addVars(range(NUM_TEAMS), name="s")

# We add an auxiliary variable z that is the minimum of the scores of all teams
z = m.addVar(name="z")

#decision variable that indicates if student i is assigned to their first choice
z1 = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="z")
print(z)

#decision variable that indicates if student i is assigned to their second choice
z2 = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="z2")

#decision variable that indicates if student i is assigned to their third choice
z3 = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="z3")


In [5]:
# Create linear objective
# The objective is to maximize z, the minimum score of any team, as well as to minimize the number of teams
# There is another part to this objective, that is currently not being used because the model below makes the total number of teams a constant (making the y variables redundant)
z_wt = 1
m.setObjective(z_wt * z - (1 - z_wt) * gp.quicksum(y[j] for j in range(NUM_TEAMS)), gp.GRB.MAXIMIZE)

In [ ]:
# Add constraints

# each student is in exactly one team
m.addConstrs(
    gp.quicksum(x[i,j] for j in range(len(students)//2)) == 1
    for i in students
)

# each team has at least 2 students
m.addConstrs(
    gp.quicksum(x[i,j] for i in students) >= 2 * y[j]
    for j in range(len(students)//2)
)

# each team has at most 3 students
m.addConstrs(
    gp.quicksum(x[i,j] for i in students) <= 3 * y[j]
    for j in range(len(students)//2)
)

# z is the minimum of the scores of all teams
BIG_M = 45
m.addConstrs(
    z <= s[j] + BIG_M * (1 - y[j])
    for j in range(len(students)//2))



'''
student_scores = []
for student in team:
    if preferences[student][0] is in team:
        add 8 to student scores list
    if preferences[student][1] is in team:
        add 4 
    if preferences[student][1] is in team:
        add 2
    
    team score is the sum of the student student_scores
'''

weights = [8,4,2]
m.addConstrs(
    s[j] == gp.quicksum(
        gp.quicksum(
            10*x[preferences[j][0],j] + 5*x[preferences[j][1],j]
        )
        for i in students
    )
    for j in range(len(students)//2)
)




# s[j] is the score of team j, where each category is weighted
m.addConstrs(
    s[j] == gp.quicksum(
        gp.quicksum(
            weight[k] * comfort[(i,categories[k])] * x[i,j]
            for k in range(len(categories))
        )
        for i in students
    )
    for j in range(len(students)//2)
)

# Add constraint that total number of teams is ceil(number of students / 3) --> this actually makes the y variables unnecessary, I think
m.addConstr(gp.quicksum([y[j] for j in range(NUM_TEAMS)]) == math.ceil(len(students) / 3))

# Add constraint for minimum coding score for each team with at least one student
MIN_CODING_SCORE = 7
m.addConstrs(gp.quicksum(comfort[(i,categories[0])] * x[i,j] for i in students) >= MIN_CODING_SCORE * y[j] for j in range(NUM_TEAMS))

# Make teams ordered by comfort
m.addConstrs(s[j] <= s[j+1] for j in range(NUM_TEAMS-1))